In [64]:
import tensorflow as tf



In [66]:
(x_train,y_train),(x_test,y_test) = tf.keras.datasets.mnist.load_data()


In [67]:
input_width = 28
input_height = 28
input_channels = 1
input_pixels = 784

n_conv1 =32
n_conv2 = 64
stride_conv1 = 1
stride_conv2 = 1

conv1_k = 5        # filter size
conv2_k = 5

max_pool1_k = 2
max_pool2_k = 2   # pooling size

n_hidden = 1024
n_out =10

input_size_to_hidden = input_width//(max_pool1_k*max_pool2_k) * input_height//(max_pool1_k*max_pool2_k) * n_conv2

In [68]:
weights = {
    
    "wc1": tf.Variable(tf.random.normal([conv1_k,conv1_k,input_channels,n_conv1])),
    "wc2": tf.Variable(tf.random.normal([conv2_k,conv2_k,n_conv1,n_conv2])),
    "wh1": tf.Variable(tf.random.normal([input_size_to_hidden,n_hidden])),
    "wo" : tf.Variable(tf.random.normal([n_hidden,n_out]))
    
}

biases = {
    "bc1" : tf.Variable(tf.random.normal([n_conv1])),
    "bc2" : tf.Variable(tf.random.normal([n_conv2])),
    "bh1" : tf.Variable(tf.random.normal([n_hidden])),
    "bho" : tf.Variable(tf.random.normal([n_out]))
    
    
    
}

In [69]:
x_train.shape

(60000, 28, 28)

In [70]:
def conv(x,weights,bias,stride=1):
    out = tf.nn.conv2d(x,weights,padding="SAME",strides=[1,stride,stride,1])
    out = tf.nn.bias_add(out,bias)
    out = tf.nn.relu(out)
    return out


def maxpooling(x,k=2):
    return tf.nn.max_pool(x,padding="SAME",ksize=[1,k,k,1],strides=[1,k,k,1])
    
    # k size deault format = [no.of examples,height,width,channels]

In [80]:
def cnn(x,weights,biases):
    x = tf.reshape(x,shape=[-1,input_height,input_width,input_channels])
    
    conv1 = conv(x,weights['wc1'],biases['bc1'],stride_conv1)
    conv1_pool = maxpooling(conv1,max_pool1_k)
    
    conv2 = conv(conv1_pool,weights['wc2'],biases['bc2'],stride_conv2)
    conv2_pool = maxpooling(conv2,max_pool2_k)
    
    hidden_input = tf.reshape(conv2_pool,shape=[-1,input_size_to_hidden])
    
    hidden_output_before_activation = tf.add(tf.matmul(hidden_input,weights['wh1']),biases['bh1'])
    
    hidden_output = tf.nn.relu(hidden_output_before_activation)
    
    output = tf.add(tf.matmul(hidden_output,weights['wo']),biases['bho'])
    
    return output

In [72]:
import tensorflow.compat.v1 as v1
tf.compat.v1.enable_eager_execution()

In [10]:
# x = v1.placeholder("float",[None,input_pixels])
# y= v1.placeholder(tf.int32,[None,n_out])
# tf.compat.v1.enable_eager_execution()
# pred = cnn(x,weights,biases)

In [ ]:
pred = cnn(x_train,weights,biases)

2024-05-27 18:44:56.964246: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 188160000 exceeds 10% of free system memory.
2024-05-27 18:44:57.068859: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 6021120000 exceeds 10% of free system memory.


In [73]:
import numpy as np

In [74]:
x_train = np.float32(x_train)


In [75]:

# cost  = tf.math.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred,lables=y))

In [77]:
optimizer = tf.optimizers.Adam(learning_rate=0.01)
# optimize = optimizer.minimize(cost)


In [134]:
batch_size =100
total_cost  = 0
for i in range(1):
    num_batches = int(x_train.shape[0]/batch_size)
    s=0
    
#     total_cost = 0
    for j in range(num_batches):
        batch_x,batch_y = x_train[s:s+batch_size],y_train[s:s+batch_size]
        batch_y_encoded = np.zeros((batch_y.size, batch_y.max()+1), dtype=int)

#replacing 0 with a 1 at the index of the original array
        batch_y_encoded[np.arange(batch_y.size),batch_y] = 1 

        s+=batch_size
#         pred = cnn(batch_x,weights,biases)
#         print(pred.shape)
        c  = tf.math.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred,labels=batch_y_encoded))
        _ = optimize = optimizer.minimize(c)
        total_cost+=c
        print(total_cost)
        
total_cost


AttributeError: 'Adam' object has no attribute 'minimize'

In [117]:
total_cost

<tf.Tensor: shape=(), dtype=float32, numpy=972314200000.0>

In [ ]:
[]

In [122]:
batch_y = y_train[0:100]

In [123]:
batch_y.shape

(100,)

In [127]:
batch_x= x_train[0:100]
batch_x.shape

(100, 28, 28)

In [130]:
# pred.shape = [100,10]
batch_y(one_hot=True)

TypeError: 'numpy.ndarray' object is not callable

In [132]:
encoded_array = np.zeros((batch_y.size, batch_y.max()+1), dtype=int)

#replacing 0 with a 1 at the index of the original array
encoded_array[np.arange(batch_y.size),batch_y] = 1 

print(encoded_array)
print(batch_y)

[[0 0 0 0 0 1 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1]
 [0 0 1 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 1 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1]
 [0 1 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0]
 [1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 1 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0

In [138]:
from tensorflow.keras.optimizers import Adam

optimizer = Adam(learning_rate=0.01)

 
    

In [141]:
optimizer.minimize(x)

AttributeError: 'Adam' object has no attribute 'minimize'

In [145]:
predictions = tf.argmax(pred,1)
predictions
print(pred.shape)
correct_labels = tf.argmax(y)

(100, 10)


NameError: name 'y' is not defined